In [2]:
# saccade_detection.py
# Requirements: numpy, pandas
# Minimal, dependency-light implementation of a velocity-based saccade detector.

import ast
import numpy as np
import pandas as pd

def parse_tuple_cell(cell):
    if pd.isna(cell) or cell == "":
        return (np.nan, np.nan)
    try:
        t = ast.literal_eval(cell)
        # some fields may be 3D; take first two if present
        if isinstance(t, (list, tuple)) and len(t) >= 2:
            return (float(t[0]), float(t[1]))
    except Exception:
        pass
    return (np.nan, np.nan)

def build_gaze_trace(df,
                     left_xy_col='left_gaze_point_on_display_area',
                     right_xy_col='right_gaze_point_on_display_area',
                     left_valid_col='left_gaze_point_validity',
                     right_valid_col='right_gaze_point_validity'):
    # parse tuples to arrays
    left = df[left_xy_col].apply(parse_tuple_cell).apply(pd.Series).rename(columns={0:'lx',1:'ly'})
    right = df[right_xy_col].apply(parse_tuple_cell).apply(pd.Series).rename(columns={0:'rx',1:'ry'})
    valid_l = df[left_valid_col].astype(float)
    valid_r = df[right_valid_col].astype(float)
    # choose binocular average where both valid; else the valid eye; else NaN
    gx = np.where((valid_l==1) & (valid_r==1), (left['lx'].values + right['rx'].values)/2,
                  np.where(valid_l==1, left['lx'].values,
                           np.where(valid_r==1, right['rx'].values, np.nan)))
    gy = np.where((valid_l==1) & (valid_r==1), (left['ly'].values + right['ry'].values)/2,
                  np.where(valid_l==1, left['ly'].values,
                           np.where(valid_r==1, right['ry'].values, np.nan)))
    return gx.astype(float), gy.astype(float)

def mad(x):
    x = np.asarray(x)
    med = np.nanmedian(x)
    return np.nanmedian(np.abs(x - med))

def detect_saccades_from_csv(csv_path,
                             ts_col='device_time_stamp',
                             ts_unit_microsec=True,
                             smoothing_window_samples=7,
                             vel_threshold_k=6.0,
                             min_duration_ms=6.0,
                             min_amplitude_norm=0.01,
                             merge_gap_ms=10.0):
    """
    Returns a pandas DataFrame of saccades with onset/offset times, duration (ms),
    amplitude (normalized units), peak velocity (norm units/sec), direction (deg),
    start/end positions.
    Parameters tuned for normalized coordinates [0..1] on display area.
    """
    df = pd.read_csv(csv_path)
    # build gaze trace
    gx, gy = build_gaze_trace(df)
    # timestamps to seconds
    ts = df[ts_col].astype(float).values
    if ts_unit_microsec:
        t = ts / 1e6
    else:
        t = ts.astype(float)
    # if timestamps monotonic but irregular, compute dt per sample
    dt = np.diff(t)
    if len(dt) == 0:
        raise ValueError("Not enough samples.")
    median_dt = np.median(dt)
    fs = 1.0 / median_dt
    # assemble DataFrame for convenience
    G = pd.DataFrame({'t': t, 'gx': gx, 'gy': gy})
    # linear interpolation of NaNs (small gaps); leave long gaps as NaN
    G['gx'] = G['gx'].interpolate(limit_direction='both')
    G['gy'] = G['gy'].interpolate(limit_direction='both')
    # smoothing: rolling median (robust) then rolling mean to preserve edges
    w = max(3, int(smoothing_window_samples) | 1)  # odd window
    G['gx_s'] = G['gx'].rolling(window=w, center=True, min_periods=1).median()
    G['gy_s'] = G['gy'].rolling(window=w, center=True, min_periods=1).median()
    # compute velocity using central difference (better than forward diff)
    # v_i = sqrt(((x_{i+1}-x_{i-1})/(t_{i+1}-t_{i-1}))^2 + ...)
    gx_arr = G['gx_s'].values
    gy_arr = G['gy_s'].values
    t_arr = G['t'].values
    v = np.full_like(gx_arr, np.nan, dtype=float)
    N = len(gx_arr)
    for i in range(1, N-1):
        dt_c = t_arr[i+1] - t_arr[i-1]
        if dt_c <= 0:
            v[i] = np.nan
            continue
        vx = (gx_arr[i+1] - gx_arr[i-1]) / dt_c
        vy = (gy_arr[i+1] - gy_arr[i-1]) / dt_c
        v[i] = np.sqrt(vx*vx + vy*vy)
    # handle ends with forward/backward diff
    v[0] = np.sqrt(((gx_arr[1]-gx_arr[0])/(t_arr[1]-t_arr[0]))**2 + ((gy_arr[1]-gy_arr[0])/(t_arr[1]-t_arr[0]))**2)
    v[-1] = np.sqrt(((gx_arr[-1]-gx_arr[-2])/(t_arr[-1]-t_arr[-2]))**2 + ((gy_arr[-1]-gy_arr[-2])/(t_arr[-1]-t_arr[-2]))**2)
    G['vel'] = v
    # robust threshold: median + k * MAD (MAD scaled like std if desired)
    vel_med = np.nanmedian(v)
    vel_mad = mad(v)
    vel_threshold = vel_med + vel_threshold_k * vel_mad
    # fallback: if MAD is 0 (very clean), use percentile
    if vel_mad == 0 or np.isnan(vel_mad):
        vel_threshold = np.nanpercentile(v[~np.isnan(v)], 95)
    # mark candidate saccade samples
    sacc_mask = (G['vel'] > vel_threshold).astype(int)
    # merge short gaps between saccades < merge_gap_ms
    if merge_gap_ms is not None and merge_gap_ms > 0:
        merge_gap_s = merge_gap_ms / 1000.0
        # find transitions
        idx = np.arange(len(sacc_mask))
        # find saccade segments
        segments = []
        in_s = False
        for i, val in enumerate(sacc_mask):
            if val and not in_s:
                start = i
                in_s = True
            if not val and in_s:
                end = i-1
                segments.append((start, end))
                in_s = False
        if in_s:
            segments.append((start, len(sacc_mask)-1))
        # try merge near segments with small gap
        merged = []
        for seg in segments:
            if not merged:
                merged.append(list(seg))
            else:
                prev = merged[-1]
                gap = (t_arr[seg[0]] - t_arr[prev[1]])
                if gap <= merge_gap_s:
                    # merge
                    prev[1] = seg[1]
                else:
                    merged.append(list(seg))
        # construct final sacc_mask from merged segments
        sacc_mask = np.zeros_like(sacc_mask)
        for a,b in merged:
            sacc_mask[a:b+1] = 1
    # enforce minimum duration and amplitude
    min_dur_s = max(0.001, min_duration_ms / 1000.0)
    # find final segments
    saccades = []
    i = 0
    N = len(sacc_mask)
    while i < N:
        if sacc_mask[i] == 1:
            start = i
            while i < N and sacc_mask[i] == 1:
                i += 1
            end = i-1
            onset_t = t_arr[start]
            offset_t = t_arr[end]
            dur = (offset_t - onset_t) * 1000.0
            if (offset_t - onset_t) < min_dur_s:
                continue
            start_pos = np.array([gx_arr[start], gy_arr[start]])
            end_pos = np.array([gx_arr[end], gy_arr[end]])
            if np.any(np.isnan(start_pos)) or np.any(np.isnan(end_pos)):
                continue
            amplitude = np.linalg.norm(end_pos - start_pos)
            if amplitude < min_amplitude_norm:
                continue
            peak_v = np.nanmax(v[start:end+1])
            # direction in degrees (0 = right, 90 = up if y increases upward)
            dx, dy = (end_pos - start_pos)
            direction_rad = np.arctan2(dy, dx)
            direction_deg = np.degrees(direction_rad)
            saccades.append({
                'onset_time_s': onset_t,
                'offset_time_s': offset_t,
                'duration_ms': dur,
                'amplitude_norm': amplitude,
                'peak_velocity_norm_per_s': peak_v,
                'start_x': start_pos[0],
                'start_y': start_pos[1],
                'end_x': end_pos[0],
                'end_y': end_pos[1],
                'direction_deg': direction_deg
            })
        else:
            i += 1
    sacc_df = pd.DataFrame(saccades)
    # also return threshold and some metadata
    meta = {'fs_est_hz': fs, 'vel_threshold': vel_threshold, 'vel_median': vel_med, 'vel_mad': vel_mad}
    return sacc_df, G, meta

In [3]:
# Example usage:
sacc_df, gaze_df, meta = detect_saccades_from_csv(r"D:\GITHUB\eye-tracking-ccs\preprocessing & analysis\eye-data-sept2025\TRIALS\krithika_trial\eye_tracking_data.csv")
print(meta); print(sacc_df.head())

{'fs_est_hz': 1199.0407674713244, 'vel_threshold': 1.5182813591392252, 'vel_median': 0.2907366313664803, 'vel_mad': 0.20459078796212415}
   onset_time_s  offset_time_s  duration_ms  amplitude_norm  \
0   1353.033202    1353.079034       45.832        1.964933   
1   1353.114035    1353.187369       73.334        0.042956   
2   1353.248201    1353.652369      404.168        1.133479   
3   1353.889869    1353.920700       30.831        0.060458   
4   1354.797368    1354.899034      101.666        0.673880   

   peak_velocity_norm_per_s   start_x   start_y     end_x     end_y  \
0                101.187020  0.273896  1.470546 -0.116674 -0.455179   
1                  8.073764 -0.113063 -0.429532 -0.117932 -0.472211   
2                234.462380 -0.109595 -0.475065  0.437607  0.517580   
3                  6.345994  0.438520  0.508938  0.494370  0.485786   
4                 13.105294  0.494216  0.492587  0.112347  1.047826   

   direction_deg  
0    -101.465041  
1     -96.507813  


In [4]:
sacc_df.columns

Index(['onset_time_s', 'offset_time_s', 'duration_ms', 'amplitude_norm',
       'peak_velocity_norm_per_s', 'start_x', 'start_y', 'end_x', 'end_y',
       'direction_deg'],
      dtype='object')

In [13]:
print(sacc_df.duration_ms.mean())
print(sacc_df.duration_ms.std())
print(sacc_df.duration_ms.max())
print(sacc_df.duration_ms.min())

53.45750800915057
48.86196033047507
622.4999999999454
6.66599999999562


In [14]:
meta

{'fs_est_hz': 1199.0407674713244,
 'vel_threshold': 1.5182813591392252,
 'vel_median': 0.2907366313664803,
 'vel_mad': 0.20459078796212415}

In [5]:
sacc_df.shape

(437, 10)

In [7]:
sacc_df

,onset_time_s,offset_time_s,duration_ms,amplitude_norm,peak_velocity_norm_per_s,start_x,start_y,end_x,end_y,direction_deg
0,1353.033202,1353.079034,45.832,1.964933,101.187020,0.273896,1.470546,-0.116674,-0.455179,-101.465041
1,1353.114035,1353.187369,73.334,0.042956,8.073764,-0.113063,-0.429532,-0.117932,-0.472211,-96.507813
2,1353.248201,1353.652369,404.168,1.133479,234.462380,-0.109595,-0.475065,0.437607,0.517580,61.133932
3,1353.889869,1353.920700,30.831,0.060458,6.345994,0.438520,0.508938,0.494370,0.485786,-22.515846
4,1354.797368,1354.899034,101.666,0.673880,13.105294,0.494216,0.492587,0.112347,1.047826,124.518508
...,...,...,...,...,...,...,...,...,...,...
432,1494.432040,1494.456206,24.166,0.080870,5.946440,0.435089,0.541315,0.495112,0.487119,-42.080046
433,1494.937874,1494.947040,9.166,0.129519,39.631907,0.499113,0.483559,0.532592,0.608677,75.019555
434,1495.127040,1495.154542,27.502,0.061844,17.625695,0.469486,0.542570,0.504525,0.491610,-55.489048
435,1496.357884,1496.383717,25.833,0.016872,11.558993,0.493807,0.516646,0.503264,0.502673,-55.912043
